<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# GitHub - Create leaderboard of contributors
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/GitHub/GitHub_Get_commits_ranking_from_repository.ipynb" target="_parent">
       <img src="https://naasai-public.s3.eu-west-3.amazonaws.com/open_in_naas.svg"/></a><br><br><a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=&template=template-request.md&title=Tool+-+Action+of+the+notebook+">Template request</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=GitHub+-+Get+commits+ranking+from+repository:+Error+short+description">Bug report</a>

**Tags:** #github #repos #commits #stats #naas_drivers #plotly #linechart #operations #analytics #dataframe #html

**Author:** [Suhas B](https://www.linkedin.com/in/suhasbrao/)


## Input

In [1]:
import pandas as pd
import plotly.express as px
from naas_drivers import github
import naas

import requests
from urllib.parse import urlencode

Note: Before executing the below line, please do add your GitHub token as a naas secret using the below line

naas.secret.add("Git_Token", <i> Put your token here</i>)

In [2]:
# Github repository url
REPO_URL = "https://github.com/jupyter-naas/awesome-notebooks"

# Github token
GITHUB_TOKEN = naas.secret.get("Git_Token")

In [3]:
# write a function to get the actor for closing an issue
def get_actor_for_closed_issue(events_url, git_obj):
    
    url = events_url
    res = requests.get(url, headers=git_obj.headers)
    try:
        res.raise_for_status()
    except requests.HTTPError as e:
        raise(e)
    
    res_json = res.json()
    if len(res_json) == 0:
        return
    
#     print(len(res_json))
    
    actor = None
    for events in res_json:
        if events["event"] == "closed":
#             print(events["actor"]["login"])
            actor= events["actor"]["login"]
    
    return actor
# get_actor_for_closed_issue("https://api.github.com/repos/jupyter-naas/awesome-notebooks/issues/1392/events")

In [91]:
def get_commits(df):
    # Exclude Github commits
    df = df[(df.COMMITTER_EMAIL.str[-10:] != "github.com")]
    
    # Groupby and count
    df = df.groupby(["AUTHOR_NAME"], as_index=False).agg({"ID": "count"})
    
    # Cleaning
    df = df.rename(columns={"ID": "NB_COMMITS"})
    return df.sort_values(by="NB_COMMITS", ascending=False).reset_index(drop=True)

df_commits_by_user = github.connect(GITHUB_TOKEN).repos.get_commits(REPO_URL, "Dr0p42")
len(df_commits_by_user)
# df_commits_by_contributor = get_commits(df_commits)
# df_commits_by_contributor.head()

92

In [4]:
def get_all_issues(repo_url):
    '''
    This function retrives all the issues of a repository and returns a 
    dataframe with the following columns:
    
    link_to_the_issue
    issue_title
    issue_number
    issue_state
    issue_creator
    issue_closed_by
    last_created_date
    last_created_time
    last_updated_date
    last_updated_time
    

    Parameters
    ----------
    repository: str:
        Repository url from Github.
        Example : "https://github.com/jupyter-naas/awesome-notebooks"
    '''
    
    git_obj = github.connect(GITHUB_TOKEN)
    repository = git_obj.get_repository_url(repo_url)
    df = pd.DataFrame()
    page, idx = 1, 0
    while True:
        params = {
                "per_page": "100",
                "page": page,
            }
        
        # Api to get open issues from github
        url = f"https://api.github.com/repos/{repository}/issues?state=all&{urlencode(params, safe='(),')}"
        res = requests.get(url, headers=git_obj.headers)
        try:
            res.raise_for_status()
        except requests.HTTPError as e:
            raise(e)
        
        res_json = res.json()
        if len(res_json) == 0:
            break
        
#         print(len(res_json))
        for issue in res_json:
            # Fetch all the issues, check node_id to see if it is an issue or PR
            
            if(issue["node_id"].startswith("I_")):
                
                df.loc[idx,'link_to_the_issue'], df.loc[idx, 'issue_number'] = issue['html_url'], issue['number']
                df.loc[idx, 'issue_title'], df.loc[idx, 'issue_state'] = issue['title'], issue['state']
   
                df.loc[idx, "issue_creator"] = issue["user"].get("login")
                
                # Create a cloumn that stores the name of the person who closed the issue
                # else if the issue is Open the column value will be None
                df.loc[idx, "issue_closed_by"] = get_actor_for_closed_issue( issue["events_url"], git_obj )
                
                df.loc[idx, 'last_created_date'] = issue.get('created_at').strip('Z').split('T')[0]
                df.loc[idx, 'last_created_time'] = issue.get('created_at').strip('Z').split('T')[-1]
                df.loc[idx, 'last_updated_date'] = issue.get('updated_at').strip('Z').split('T')[0]
                df.loc[idx, 'last_updated_time'] = issue.get('updated_at').strip('Z').split('T')[-1]
                
                
                idx +=1
        page+=1
    return df



In [8]:
df_all_issues = get_all_issues(REPO_URL)
print("Total open Issues fetched:", len(df_all_issues))

Total open Issues fetched: 827


In [9]:
df_all_issues.head()

,link_to_the_issue,issue_number,issue_title,issue_state,issue_creator,issue_closed_by,last_created_date,last_created_time,last_updated_date,last_updated_time
0,https://github.com/jupyter-naas/awesome-notebo...,1394.0,Azure - Download files in blob storage,open,FlorentLvr,NaN,2023-01-31,15:29:14,2023-01-31,15:30:32
1,https://github.com/jupyter-naas/awesome-notebo...,1392.0,Azure - Upload files in blob storage,open,FlorentLvr,NaN,2023-01-31,10:47:27,2023-01-31,10:49:00
2,https://github.com/jupyter-naas/awesome-notebo...,1391.0,LinkedIn - Get profile information: Error shor...,open,JJayasri,NaN,2023-01-31,09:20:41,2023-01-31,09:20:41
3,https://github.com/jupyter-naas/awesome-notebo...,1389.0,FEC - Display Balance Sheet with Treemap,closed,FlorentLvr,FlorentLvr,2023-01-30,16:20:26,2023-01-31,09:59:19
4,https://github.com/jupyter-naas/awesome-notebo...,1388.0,Plotly - Create Treemap,closed,FlorentLvr,FlorentLvr,2023-01-30,16:18:33,2023-01-31,15:18:54


In [109]:
def get_issue_comments(repo_url):
    git_obj = github.connect(GITHUB_TOKEN)
    repository = git_obj.get_repository_url(repo_url)
    df = pd.DataFrame()
    page, idx = 1, 0
    while True:
        params = {
                "per_page": "100",
                "page": page,
            }
        
        # Api to get open issues from github
       
        url = f"https://api.github.com/repos/{repository}/issues/comments?{urlencode(params, safe='(),')}"
        res = requests.get(url, headers=git_obj.headers)
        try:
            res.raise_for_status()
        except requests.HTTPError as e:
            raise(e)
        
        res_json = res.json()
        if len(res_json) == 0:
            break
        
#         print(len(res_json))
        for issue_comment in res_json:
            df.loc[idx, "comment_id"] = issue_comment["id"]
            df.loc[idx, "issue_url"] = issue_comment["issue_url"]
            df.loc[idx, "comment_by"] = issue_comment["user"]["login"]
            df.loc[idx, "comment_body"] = issue_comment["body"]
            
            idx +=1
        page += 1
        
    return df

In [110]:
df_all_comments = get_issue_comments(REPO_URL)
df_all_comments.head()

In [113]:
def get_contributors_details(repo_url):
    git_obj = github.connect(GITHUB_TOKEN)
    repository = git_obj.get_repository_url(repo_url)
    df = pd.DataFrame()
    page, idx = 1, 0
    while True:
        params = {
                "per_page": "100",
                "page": page,
            }
        url = f"https://api.github.com/repos/{repository}/contributors?{urlencode(params, safe='(),')}"
        res = requests.get(url, headers=git_obj.headers)
        try:
            res.raise_for_status()
        except requests.HTTPError as e:
            raise(e)
        
        res_json = res.json()
        if len(res_json) == 0:
            break
        
        for contributor in res_json:
            df.loc[idx, "contributor"] = contributor["login"]
            # create a column to store number of issues created by a contributor
            df.loc[idx, "issues_created"] = len(df_all_issues.loc[df_all_issues["issue_creator"]==contributor["login"]])
            # create a column to store number of issues closed by a contributor
            
            df.loc[idx, "issues_closed"] = len(df_all_issues.loc[df_all_issues["issue_closed_by"]==contributor["login"]])
            
            # Need to get df for issue comments and PR comments
            df.loc[idx, "issue_pr_comments"] = len(df_all_comments.loc[df_all_comments["comment_by"] == contributor["login"]])
            
            # add a column to store number of commits made by a contributor
            df.loc[idx, "commits"] = len(git_obj.repos.get_commits(REPO_URL, contributor["login"]))
            idx +=1
        
        page += 1
    
    return df

In [114]:
df = get_contributors_details(REPO_URL)

In [117]:
df.loc[df["contributor"] == "FlorentLvr"]

,contributor,issues_created,issues_closed,issue_pr_comments,commits
0,FlorentLvr,350.0,160.0,284.0,265.0


In [6]:
def get_issues_leaders(df):
    df = df.groupby(["issue_creator"], as_index=False).agg({"issue_title":"count"})
    df = df.rename(columns={"issue_title":"Issues_Created", "issue_creator":"Creator"})
    
    return df.sort_values(by="Issues_Created", ascending=False).reset_index(drop=True)


In [7]:
# df = get_commits(df_commits)
df = get_issues_leaders(df_all_issues)
df.head()

,Creator,Issues_Created
0,FlorentLvr,349
1,Dr0p42,293
2,jravenel,105
3,Valpiquard,17
4,SanjuEpic,11


In [17]:
len(df_all_issues.loc[df_all_issues["issue_creator"]=="FlorentLvr"])

349